Treinando uma rede neural usando pytorch
========================================



## Introdução



O notebook anterior foi o *season finale* da construção da rede neural usando Python puro. Este notebook é o epílogo.

Aqui veremos como podemos construir e treinar uma rede neural usando `pytorch`, um módulo especializado em redes neurais artificiais.

Motivos para usarmos o `pytorch`:

1.  A `MLP` que criamos em Python puro não é otimizada. Tente treinar uma rede com uns 100 neurônios em uma camada e verá como ela é bem lenta! `pytorch` é escrito por diversos programadores que além de primar pelas contas estarem corretas, eles fazem de tudo para que elas ocorram de forma eficiente.

2.  `pytorch` já tem praticamente tudo que precisamos implementado! Quer calcular o gradiente local da função arco tangente? Ele já tem! Quer usar a função de ativação ReLU? Ele já tem!

3.  `pytorch` tem suporte a treinar uma rede neural usando processamento gráfico (GPU). Isso acelera muito o treino de redes neurais artificiais complexas. Temos GPUs para usarmos no HPC da Ilum!

4.  `pytorch` é usado tanto na academia quanto no mundo corporativo. Junto com `tensorflow` são os dois módulos estado da arte para o projeto e treino de redes neurais.



## Objetivo



Treinar uma rede neural artificial tipo Multilayer Perceptron usando `pytorch`.



## Importações



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

## Código e discussão



### Dados



Vamos treinar nossa rede neural com nosso velho amigo, o dataset de diamantes! Como todo processo de treino de aprendizado de máquina, precisamos reservar um conjunto de dados para treino e outro para teste.



In [2]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
DATASET_NAME = "diamonds"
FEATURES = ["carat", "depth", "table", "x", "y", "z"]
TARGET = ["price"]

df = sns.load_dataset(DATASET_NAME)

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1)
X_teste = df_teste.reindex(FEATURES, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1)

Redes neurais costumam se dar bem quando os dados estão entre $[0, 1]$ ou entre $[-1, 1]$. Redes neurais costumam não se dar bem quando os dados estão em escalas muito diferentes. Por conta disso, normalizar os dados antes de prosseguir é recomendado! Vamos usar o `MinMaxScaler` do `scikit-learn` que aprendemos semestre passado.



In [3]:
normalizador_x = MinMaxScaler()
normalizador_y = MinMaxScaler()

normalizador_x.fit(X_treino)
normalizador_y.fit(y_treino)

X_treino = normalizador_x.transform(X_treino)
y_treino = normalizador_y.transform(y_treino)
X_teste = normalizador_x.transform(X_teste)
y_teste = normalizador_y.transform(y_teste)

### Tensores



Na nossa rede neural feita em Python puro, a base de tudo era a classe `Valor`. Para o `pytorch`, a base de tudo são os tensores! Tensores são objetos que armazenam dados de forma similar aos arrays de `numpy`, porém registram todas as operações realizadas para o cálculo do gradiente local (assim como nós fizemos com a classe `Valor`).



In [4]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)
X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

Vamos checar os dados para ver como estão.



In [5]:
print(X_treino)
print()
print(y_treino)

tensor([[0.0686, 0.5611, 0.2885, 0.4777, 0.0876, 0.1022],
        [0.0624, 0.5250, 0.2500, 0.4730, 0.0871, 0.0994],
        [0.0478, 0.4917, 0.2692, 0.4572, 0.0829, 0.0934],
        ...,
        [0.1185, 0.5333, 0.2692, 0.5410, 0.0995, 0.1142],
        [0.0166, 0.5139, 0.2885, 0.3929, 0.0708, 0.0811],
        [0.0333, 0.5361, 0.2692, 0.4209, 0.0776, 0.0890]])

tensor([[0.0676],
        [0.0586],
        [0.0434],
        ...,
        [0.1527],
        [0.0172],
        [0.0216]])


### Criando a rede neural



Para criar uma rede neural usando `pytorch` é necessário criar uma classe. Observe aqui que a classe criada é baseada na classe `nn.Module` do `pytorch`. Classes podem ser baseadas em outras classes! Trata-se da característica de herança das classes.

Observe que definimos as camadas da rede neural dentro de um objeto `nn.Sequential`.

Observe também o método `forward`. Este funciona de forma similar ao dunder `__call__` que vimos anteriormente. O `pytorch` requer que usemos o `forward` e não o `__call__`, então aceitamos e seguimos em frente.



In [6]:
class MLP(nn.Module):
    def __init__(
        self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets
    ):
        # Temos que inicializar a classe mãe
        super().__init__()

        # Definindo as camadas da rede
        self.camadas = nn.Sequential(
                    nn.Linear(num_dados_entrada, neuronios_c1),
                    nn.ReLU(),
                    nn.Linear(neuronios_c1, neuronios_c2),
                    nn.ReLU(),
                    nn.Linear(neuronios_c2, num_targets),
                )

    def forward(self, x):
        """Esse é o método que executa a rede do pytorch."""
        x = self.camadas(x)
        return x

Agora podemos criar a nossa rede neural!



In [7]:
NUM_DADOS_DE_ENTRADA = X_treino.shape[1]
NUM_DADOS_DE_SAIDA = y_treino.shape[1]
NEURONIOS_C1 = 50
NEURONIOS_C2 = 20

minha_MLP = MLP(NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA)

E podemos checar os parâmetros internos dela!



In [8]:
for p in minha_MLP.parameters():
    print(p)

Parameter containing:
tensor([[ 0.1676,  0.3520, -0.2806,  0.3828, -0.2962,  0.0678],
        [-0.3010, -0.0193,  0.3626, -0.2180,  0.0800, -0.0802],
        [-0.1659,  0.0675, -0.2661, -0.3592, -0.0450,  0.2890],
        [-0.2231,  0.3222, -0.2057,  0.1826, -0.2221,  0.0453],
        [ 0.2000,  0.0704, -0.2288, -0.1115, -0.3646,  0.3436],
        [ 0.1066,  0.3665, -0.2236, -0.1703,  0.2116,  0.1508],
        [ 0.0967,  0.0616,  0.0369, -0.1711,  0.2434,  0.1294],
        [-0.0832,  0.3892,  0.3469, -0.3567,  0.0990, -0.0522],
        [ 0.0348,  0.1472,  0.0145,  0.0815, -0.3447,  0.1323],
        [-0.2732,  0.1989,  0.1228, -0.1344,  0.3103, -0.3461],
        [-0.1152,  0.1804, -0.3313,  0.3229,  0.0968,  0.1301],
        [ 0.2508, -0.1244, -0.3011, -0.3864, -0.2760,  0.3121],
        [ 0.1552, -0.2478,  0.2292, -0.1832,  0.2512,  0.2953],
        [ 0.0921,  0.3911,  0.3183,  0.4018, -0.3275, -0.1546],
        [-0.4061,  0.0387,  0.2175, -0.2672, -0.0744,  0.1433],
        [-0.0485, 

Também podemos realizar uma previsão, mas ela provavelmente será bem ruim!



In [9]:
y_prev = minha_MLP(X_treino)
y_prev

tensor([[0.1895],
        [0.1900],
        [0.1887],
        ...,
        [0.1926],
        [0.1829],
        [0.1853]], grad_fn=<AddmmBackward0>)

### A função de perda e o otimizador



O `pytorch` já vem com diversas funções de perda já implementadas. A que computa o erro quadrático médio, por exemplo, já está pronta!

Fora isso, precisamos definir o nosso otimizador. O otimizador é quem cuida de atualizar os parâmetros da nossa rede. Nós implementamos na nossa rede em Python puro a descida do gradiente. Aqui usaremos um otimizador que é uma modificação da descida do gradiente, chamado `Adam`. Trata-se, simplificadamente, de uma descida do gradiente com taxa de aprendizado individualizada para cada parâmetro e que se altera ao longo do aprendizado. `Adam` é um poderosíssimo otimizador! Não tem porque não usarmos ele neste caso.



In [10]:
TAXA_DE_APRENDIZADO = 0.001

# função perda será o erro quadrático médio
fn_perda = nn.MSELoss()

# otimizador será o Adam, um tipo de descida do gradiente
otimizador = optim.Adam(minha_MLP.parameters(), lr=TAXA_DE_APRENDIZADO)

### O treino da rede



Para avisar o `pytorch` que iremos treinar a rede, vamos invocar o método `train` da nossa rede. Toda rede recém-criada já está no modo treino, mas aqui deixarei explícito para ficar bem claro.



In [11]:
minha_MLP.train()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Observe que o ciclo do treino tem as mesmas etapas que vimos no notebook anterior, nada de novo! Aqui basta observar como executamos essas etapas no `pytorch`.



In [17]:
NUM_EPOCAS = 10000

y_true = y_treino

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = minha_MLP(X_treino)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(y_pred, y_true)

    # backpropagation
    loss.backward()

    # atualiza parâmetros
    otimizador.step()

    # mostra resultado
    print(epoca, loss.data)

0 tensor(0.0056)
1 tensor(0.0056)
2 tensor(0.0056)
3 tensor(0.0056)
4 tensor(0.0056)
5 tensor(0.0056)
6 tensor(0.0056)
7 tensor(0.0056)
8 tensor(0.0056)
9 tensor(0.0056)
10 tensor(0.0056)
11 tensor(0.0056)
12 tensor(0.0056)
13 tensor(0.0056)
14 tensor(0.0056)
15 tensor(0.0056)
16 tensor(0.0056)
17 tensor(0.0056)
18 tensor(0.0056)
19 tensor(0.0056)
20 tensor(0.0056)
21 tensor(0.0056)
22 tensor(0.0056)
23 tensor(0.0056)
24 tensor(0.0056)
25 tensor(0.0056)
26 tensor(0.0056)
27 tensor(0.0056)
28 tensor(0.0056)
29 tensor(0.0056)
30 tensor(0.0056)
31 tensor(0.0056)
32 tensor(0.0056)
33 tensor(0.0056)
34 tensor(0.0056)
35 tensor(0.0056)
36 tensor(0.0056)
37 tensor(0.0056)
38 tensor(0.0056)
39 tensor(0.0056)
40 tensor(0.0056)
41 tensor(0.0056)
42 tensor(0.0056)
43 tensor(0.0056)
44 tensor(0.0056)
45 tensor(0.0056)
46 tensor(0.0056)
47 tensor(0.0056)
48 tensor(0.0056)
49 tensor(0.0056)
50 tensor(0.0056)
51 tensor(0.0056)
52 tensor(0.0056)
53 tensor(0.0056)
54 tensor(0.0056)
55 tensor(0.0056)
56

Após o treino, podemos checar a performance da nossa rede. Observe a linha que inicia com `with`. Tudo que estiver no bloco do `with torch.no_grad()` é computado normalmente, porém o grafo computacional e os gradientes locais não são computados. Use isso sempre que for realizar contas com tensores fora do treino!!



In [18]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_treino)
    y_pred = minha_MLP(X_treino)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[1577.99996901] [1570.39735317]
[1409.99999118] [1550.44870329]
[1128.99997044] [1117.0046525]
[12991.99967575] [11915.91609573]
[14720.0001564] [14811.95640564]
[2938.99994564] [2646.94589043]
[6635.00014687] [7512.29425049]
[11215.99973297] [9974.32743454]
[1040.99999189] [1592.31428528]
[5040.00003624] [5536.41892052]
[9541.99988747] [8025.01943588]
[3792.99988461] [5757.84495163]
[1002.0000155] [736.47601509]
[2542.00005341] [1714.36266613]
[6788.00001717] [6329.29531097]
[9386.00025749] [7723.67533112]
[1245.99996853] [1769.03634548]
[997.00000262] [1198.57295227]
[708.00000262] [776.20629406]
[1435.99997544] [1483.33803797]
[2590.00006676] [2675.19748592]
[1392.99997497] [1724.23591042]
[11755.99946976] [10093.34436035]
[2007.99997377] [1558.06813145]
[2707.99998474] [2617.29391098]
[631.00000417] [727.37200356]
[652.99999881] [1073.85830212]
[2492.00006247] [2538.62160301]
[594.99999416] [560.98070335]
[760.0000056] [1114.59456444]
[1179.00003028] [1099.05060577]
[1789.9999361] 

### O teste da rede



Vamos supor que você já testou diversas arquiteturas de rede e está pronto para usar os dados de teste. Nesta etapa precisamos indicar para o `pytorch` que o treino acabou e estamos na etapa de avaliação. Fazemos isso rodando o método `eval`.



In [22]:
minha_MLP.eval()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Agora podemos realizar o teste!



In [23]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_teste)
    y_pred = minha_MLP(X_teste)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[523.00000858] [850.79210472]
[628.00000334] [619.93987179]
[442.00000328] [746.77370167]
[2284.99997044] [2445.76947737]
[1040.99999189] [801.93080425]
[809.99999654] [1232.7818985]
[3049.99987316] [3111.72122383]
[720.00000596] [690.75283146]
[877.99999249] [572.13306236]
[1292.00000429] [1679.11218262]
[7056.00004578] [5156.62520218]
[4176.99999142] [5752.65021896]
[13701.00020599] [14430.51177216]
[2948.0000515] [3890.82804489]
[16732.99985504] [14434.85872269]
[1815.00000048] [1930.98175049]
[16220.00016022] [12168.848423]
[5967.00019073] [4690.32297897]
[473.00000042] [657.56502724]
[1078.9999795] [1038.77897358]
[452.00000319] [637.49552155]
[801.99998283] [814.83302307]
[2458.99994993] [2443.38667488]
[2492.99998236] [2824.61081886]
[582.00000203] [724.69856834]
[1405.9999671] [1446.11941004]
[8715.99991035] [11585.97780991]
[7634.00004578] [6814.393713]
[506.99999839] [692.54899406]
[665.99999094] [769.09771252]
[1815.00000048] [1592.97423792]
[707.00001383] [727.02803993]
[59

E, finalmente, computar alguma métrica para medir a performance do nosso modelo.



In [21]:
RMSE = mean_squared_error(y_true, y_pred, squared=False)
print(f'Loss do teste: {RMSE}')

Loss do teste: 1366.8323789559004


## Conclusão



## Playground

